# Otomatik Araç Kullanım Örneği


## Gerekli Paketleri İçe Aktar


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## İstemciyi Oluşturma

Bu örnekte, LLM'e erişim için [GitHub Modelleri](https://aka.ms/ai-agents-beginners/github-models) kullanacağız.

`model`, `gpt-4o-mini` olarak tanımlanmıştır. GitHub Modelleri pazarında bulunan başka bir modele geçiş yaparak farklı sonuçları görmeyi deneyin.

Hızlı bir test olarak, basit bir istem çalıştıracağız - `Fransa'nın başkenti nedir`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Fonksiyonları Tanımlama

Bu örnekte, ajana mevcut tatil destinasyonlarının bir listesini ve bunların uygunluk durumlarını içeren bir fonksiyona erişim sağlayacağız.

Bunu, bir seyahat acentesinin örneğin bir seyahat veritabanına erişimi olduğu bir senaryo olarak düşünebilirsiniz.

Bu örneği incelerken, ajanın çağırabileceği yeni fonksiyonlar ve araçlar tanımlamayı denemekten çekinmeyin.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Fonksiyon Aracını Tanımlama  
Ajanın `vacation_destinations` öğesini bir `FunctionTool` olarak kullanabilmesi için, bunu bir Fonksiyon Aracı olarak tanımlamamız gerekiyor.  

Ayrıca, kullanıcının talep ettiği görevle ilgili olarak bu aracın ne için kullanıldığını ajanın anlamasına yardımcı olacak bir açıklama da sağlayacağız.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Ajanı Tanımlama

Şimdi aşağıdaki kodda ajanı oluşturabiliriz. Kullanıcılara tatil destinasyonları bulmalarında nasıl yardımcı olacağını açıklamak için `system_message` tanımlıyoruz.

Ayrıca `reflect_on_tool_use` parametresini true olarak ayarlıyoruz. Bu, aracın çağrısına verilen yanıtı alıp doğal dil kullanarak yanıt göndermemizi sağlar.

Farkı görmek için parametreyi false olarak ayarlayabilirsiniz.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Ajanı Çalıştırma

Şimdi, Tokyo'ya bir gezi yapmayı talep eden ilk kullanıcı mesajıyla ajanı çalıştırabiliriz.

Ajanın, şehrin uygunluğuna nasıl yanıt verdiğini görmek için bu şehir hedefini değiştirebilirsiniz.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Orijinal belgenin kendi dilindeki hali, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
